Connect to T4 GPU!!! Takuto was here

Notebooks: most code was copied from the first notebook, except the first cell for import which was copied from the second notebook (since the first notebook caused errors)

1. Finetuning Llama 3.1: https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-Alpaca.ipynb

2. GRPO (R1 reasoning) with Llama 3.1: https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb#scrollTo=GPgXROedCcqP


Useful links:
- https://docs.unsloth.ai/basics/continued-pretraining
- https://docs.unsloth.ai/get-started/fine-tuning-guide

In [ ]:
# !pip install --no-deps unsloth vllm
# import sys, re, requests; modules = list(sys.modules.keys())
# for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
# !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
# !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

# # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
# f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
# with open("vllm_requirements.txt", "wb") as file:
#     file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
# !pip install -r vllm_requirements.txt

%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from unsloth import FastLanguageModel
import torch
import os
import numpy as np

max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!

model_name = "unsloth/Meta-Llama-3.1-8B" # default
# model_name = "./model" # loading a pretrained saved model

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = None,  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    load_in_4bit = True,    # Use 4bit quantization to reduce memory usage. Can be False.
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Rank: Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.4.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
prompt = """
### Instruction:
Below is a math problem

### Input:
{}

### Response:
{}
"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_func_gsm8k(examples):
    question = examples['question']
    answer = examples['answer']
    texts = []

    for q, a in zip(question, answer):
        text = prompt.format(q, a) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

def formatting_func_MATH(examples):
    question = examples['problem']
    level = examples['level']
    answer = examples['solution']
    texts = []

    for q, a in zip(question, answer):
        text = prompt.format(q, a) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }


from datasets import load_dataset
gsm8k = load_dataset("openai/gsm8k", "main")
gsm8k_train = gsm8k['train']
gsm8k_test = gsm8k['test']
gsm8k_train = gsm8k_train.map(formatting_func_gsm8k, batched=True)
gsm8k_test = gsm8k_test.map(formatting_func_gsm8k, batched=True)

MATH = load_dataset("nlile/hendrycks-MATH-benchmark")
MATH_train = MATH['train']
MATH_test = MATH['test']
MATH_train = MATH_train.map(formatting_func_MATH, batched=True)
MATH_test = MATH_test.map(formatting_func_MATH, batched=True)

# dataset = load_dataset("openai/gsm8k", "main", split="train")
# dataset = dataset.map(formatting_func_gsm8k, batched=True)


README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/210k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
from datasets import concatenate_datasets
combined_dataset_train= concatenate_datasets([MATH_train,gsm8k_train])
combined_dataset_train = combined_dataset_train.shuffle(seed=42)


combined_dataset_test= concatenate_datasets([MATH_test,gsm8k_test])
combined_dataset_test = combined_dataset_test.shuffle(seed=42)





In [ ]:
#the fields that are not in common simply get mapped to None. We are only interested in the text field so this does not matter for now
combined_dataset_test[10]

{'problem': None,
 'solution': None,
 'answer': 'Each sandwich serves 4 people and he had 20 people in total to make food for so 20/4 = <<20/4=5>>5 sandwiches\n1 pound of meat cost $7.00 per sandwich and he had to make 5 so 7*5 = $<<7*5=35.00>>35.00 for meat\n1 pound of cheese cost $3.00 per sandwich and he had to make 5 so 3*5 = $<<3*5=15.00>>15.00 for cheese\nWhen you add the meat and cheese together, he would spend 35+15 = $<<35+15=50.00>>50.00\n#### 50',
 'subject': None,
 'level': None,
 'unique_id': None,
 'text': '\n### Instruction:\nBelow is a math problem\n\n### Input:\nTyson decided to make muffaletta sandwiches for the big game.  Each sandwich required 1 pound each of meat and cheese and would serve 4 people.  There would be 20 people in total watching the game.  The meat cost $7.00 per pound and the cheese cost $3.00 per pound.  How much money would he spend on the meat and cheese to make enough sandwiches to serve 20 people?\n\n### Response:\nEach sandwich serves 4 people 

In [ ]:
len(combined_dataset_train)

19473

In [ ]:
len(gsm8k_train)+len(MATH_train)

19473

In [ ]:
len(MATH_test)

500

In [ ]:
from trl import SFTTrainer, SFTConfig
# from peft import LoraConfig
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    # train_dataset = gsm8k_train,
    # eval_dataset = gsm8k_test,
    # train_dataset = MATH_train,
    # eval_dataset = MATH_test,
    train_dataset=combined_dataset_train,
    eval_dataset= combined_dataset_test,

    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 4, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/19473 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/1819 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 19,473 | Num Epochs = 4 | Total steps = 9,736
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


In [ ]:
# !zip -r outputs.zip outputs

# save the training progress (loss vs steps)
plot_data = list(map(lambda e: (e['step'], e['loss']), trainer.state.log_history[:-1]))
steps, loss = zip(*plot_data) # steps and loss are tuples

np.savez("training.npz", steps=steps, loss=loss)

# save the LORA updated weights (do we need to save the tokenizer too?)
model.save_pretrained_merged("model", tokenizer, save_method="lora")


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... Done.


In [ ]:
!zip -r model.zip model training.npz

save_name = input(prompt="Save name: ")
import os
from google.colab import files

os.rename("model.zip", f"{save_name}.zip")
files.download(f'{save_name}.zip')

  adding: model/ (stored 0%)
  adding: model/adapter_model.safetensors (deflated 7%)
  adding: model/special_tokens_map.json (deflated 71%)
  adding: model/tokenizer.json (deflated 85%)
  adding: model/tokenizer_config.json (deflated 96%)
  adding: model/adapter_config.json (deflated 56%)
  adding: model/README.md (deflated 66%)
  adding: training.npz (deflated 73%)
Save name: combined5


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# todo rewrite this

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

sample = gsm8k_test[int(np.random.randint(0, gsm8k_test.num_rows))]
# sample = MATH_test[int(np.random.randint(0, MATH_test.num_rows))]

inputs = tokenizer(
[
    prompt.format(
        f"{sample['question']}", # instruction
        # f"{sample['problem']}", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True)
print(tokenizer.batch_decode(outputs)[0])

print(f"\n\nSample Answer: {sample['answer']}")
# print(f"\n\nSample Answer: {sample['solution']}")

<|begin_of_text|>
### Instruction:
Below is a math problem

### Input:
Ryan’s allowance is $6 each week he completes his chores. He did his chores for 3 weeks. Then he bought ice cream cones for himself and 3 friends at $1.25 each. Now they all want to go to the movies and tickets cost $6.50 each. How many movie tickets can Ryan buy?

### Response:

Ryan has earned $6 x 3 = $<<6*3=18>>18.
He spent $1.25 x 4 = $<<1.25*4=5>>5 on ice cream cones.
He has $18 - 5 = $<<18-5=13>>13 left.
He can buy $13 / $6.50 = <<13/6.5=2>>2 tickets.
#### 2
<|end_of_text|>


Sample Answer: Ryan earned $6 x 3 weeks of chores = $<<6*3=18>>18.
He bought 1 for himself + 3 for his friends = <<1+3=4>>4 ice creams.
The ice cream cost $1.25 x 4 = $<<1.25*4=5>>5.
Ryan has $18 - $5 = $<<18-5=13>>13 left.
With the money he has left, Ryan can buy $13 / $6.50 = <<13/6.50=2>>2 movie tickets.
#### 2
